In [ ]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime, timedelta
import time  # For sleep function
from zerodha import Zerodha
import threading  # For threading
import json  # For JSON serialization
import pandas_market_calendars as mcal  # For market calendar

warnings.filterwarnings("ignore")

# Initialize Zerodha and login
kite = Zerodha(user_id='AI2416', password='Vinay@14541228', twofa='6TWJRKAU6TYJ3O2U7EYAIJEKPS3BDQBD')
kite.login()

# Load Nifty 500 symbols from CSV
equity_details = pd.read_csv("ind_nifty500list.csv")
nifty_500_symbols = equity_details['Symbol'].tolist()

# Fetch all instruments once to avoid repeated API calls
instruments = kite.instruments("NSE")

# Initialize investment parameters
initial_fund = 2000000  # Total available fund
portfolio_size = 20
investment_per_stock = initial_fund // portfolio_size  # Fixed investment per stock

stop_loss_percent = 8
volume_multiplier = 5
stock_selection = 5

# Create or load logs
log_file = 'trade_log.csv'
summary_file = 'summary_statistics.csv'
breakout_stocks_file = 'breakout_stocks.json'  # JSON file to store breakout stocks and days to check

try:
    trade_log = pd.read_csv(log_file)
except FileNotFoundError:
    trade_log = pd.DataFrame(columns=[
        'Symbol', 'Entry Date', 'Entry Time', 'Buy Price', 'Shares', 'Buy Value', 'Sell Date', 
        'Sell Price', 'Sell Value', 'Profit/Loss', 'ROI', 'High Value', 'Low Value', 'Max Drawdown', 
        'Exit Reason', 'SMA_198', 'Status'  # Added Status field
    ])

try:
    summary_stats = pd.read_csv(summary_file).iloc[0].to_dict()
except FileNotFoundError:
    summary_stats = {
        "Total Profit/Loss": 0.0,
        "Return (%)": 0.0,
        "CAGR (%)": 0.0,
        "Total Trades": 0,
        "Winning Trades": 0,
        "Winning Percentage (%)": 0.0,
        "Max Drawdown": 0.0,
    }

try:
    with open(breakout_stocks_file, 'r') as f:
        breakout_stocks_today = json.load(f)
except FileNotFoundError:
    breakout_stocks_today = {}

# NSE market calendar
nse = mcal.get_calendar('NSE')

# Function to check if a date is a trading day
def is_trading_day(date):
    schedule = nse.schedule(start_date=date, end_date=date)
    return not schedule.empty

# Function to fetch the next trading day
def next_trading_day(date):
    schedule = nse.schedule(start_date=date + timedelta(days=1), end_date=date + timedelta(days=10))
    next_day = schedule.index[0].date()
    return next_day

# Function to fetch historical data
def fetch_data(symbol, interval='day'):
    try:
        instrument_token = next((instrument['instrument_token'] for instrument in instruments if instrument['tradingsymbol'] == symbol), None)
        if instrument_token:
            # print(f"Fetching data for {symbol} for {interval} interval")
            # Adjusted to fetch data for the last 200 days
            to_date = datetime.now()
            from_date = to_date - timedelta(days=300)
            data = kite.historical_data(instrument_token, from_date, to_date, interval=interval)
            df = pd.DataFrame(data)
            if not df.empty:
                df['SMA_66'] = df['close'].rolling(window=66).mean()
                df['SMA_198'] = df['close'].rolling(window=198).mean()
                df['Max_Volume_132'] = df['volume'].rolling(window=132).max()
                df['AVG_Volume_10'] = df['volume'].ewm(span=10, adjust=False).mean()
                df['Symbol'] = symbol
                df['Date'] = pd.to_datetime(df['date']).dt.date
                return df
            return None
        else:
            print(f"No instrument found for symbol {symbol}")
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Function to fetch 15-minute data
def fetch_15min_data(symbol, date):
    try:
        instrument_token = next((instrument['instrument_token'] for instrument in instruments if instrument['tradingsymbol'] == symbol), None)
        if instrument_token:
            data_15min = kite.historical_data(instrument_token, date, date + timedelta(days=1), '15minute')
            return pd.DataFrame(data_15min)
        return None
    except Exception as e:
        print(f"Error fetching 15min data for {symbol}: {e}")
        return None

# Function to execute daily routine
def daily_routine():
    global trade_log, summary_stats, breakout_stocks_today

    current_date = datetime.now().date()
    print(f"Processing date: {current_date.strftime('%Y-%m-%d')}")

    # Fetch data for all Nifty 500 stocks
    all_data = []
    for symbol in nifty_500_symbols:
        try:
            df = fetch_data(symbol)
            if df is not None and not df.empty:
                all_data.append(df)
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")

    # Combine all data into a single DataFrame
    data = pd.concat(all_data, ignore_index=True)

    # Filter data for the current date
    current_data = data[data['Date'] == current_date]

    # Volume breakout condition
    breakout_condition = (current_data['volume'] > volume_multiplier * current_data['AVG_Volume_10']) & \
                         (current_data['volume'] >= current_data['Max_Volume_132']) & \
                         (current_data['close'] > current_data['SMA_66']) & (current_data['close'] > current_data['SMA_198'])

    today_breakout_stocks = current_data[breakout_condition].nlargest(stock_selection, 'volume')

    # Update breakout stocks for today
    new_breakout_stocks = today_breakout_stocks['Symbol'].tolist()
    for stock in new_breakout_stocks:
        breakout_stocks_today[stock] = {
            'days_to_check': 2,
            'breakout_date': str(current_date),
            'highs': {}
        }

    # Update breakout stock highs
    for stock in breakout_stocks_today:
        date_to_check = datetime.strptime(breakout_stocks_today[stock]['breakout_date'], '%Y-%m-%d').date()
        for day_count in range(3):
            if not is_trading_day(date_to_check):
                date_to_check = next_trading_day(date_to_check - timedelta(days=1))
            df_15min = fetch_15min_data(stock, date_to_check)
            if df_15min is not None and not df_15min.empty:
                breakout_stocks_today[stock]['highs'][str(date_to_check)] = df_15min['high'].max()
            date_to_check = next_trading_day(date_to_check)

    # Store updated breakout stocks and days to check in a file
    with open(breakout_stocks_file, 'w') as f:
        json.dump(breakout_stocks_today, f, indent=4)
    print(f"Breakout stocks updated: {breakout_stocks_today}")

    # Allocate investment to each breakout stock
    stocks_to_remove = []  # Keep track of stocks to remove after buying
    for symbol, details in breakout_stocks_today.items():
        previous_day_high = None
        df_15min = fetch_15min_data(symbol, current_date)
        if df_15min is not None and not df_15min.empty:
            first_15min_high = df_15min.iloc[0]['high']

            # Check if today's breakout or previous days' breakout
            if details['days_to_check'] == 2:  # Today's breakout
                # Check if any subsequent candle breaks the high of the first 15-minute candle
                breakout_candle = df_15min[df_15min['high'] > first_15min_high]
                if not breakout_candle.empty:
                    breakout_candle_close = breakout_candle.iloc[0]['close']
                    shares = np.floor(investment_per_stock / breakout_candle_close)
                    buy_value = round(shares * breakout_candle_close, 2)
                    summary_stats["Total Profit/Loss"] -= buy_value
                    
                    # Log the buy action
                    trade_log = trade_log.append({
                        'Symbol': symbol, 
                        'Entry Date': current_date, 
                        'Entry Time': breakout_candle.iloc[0]['date'], 
                        'Buy Price': breakout_candle_close, 
                        'Shares': shares, 
                        'Buy Value': buy_value, 
                        'Sell Date': np.nan, 
                        'Sell Price': np.nan, 
                        'Sell Value': np.nan, 
                        'Profit/Loss': np.nan, 
                        'ROI': np.nan, 
                        'High Value': breakout_candle.iloc[0]['high'], 
                        'Low Value': breakout_candle.iloc[0]['low'], 
                        'Max Drawdown': np.nan, 
                        'Exit Reason': np.nan, 
                        'SMA_198': current_data[current_data['Symbol'] == symbol]['SMA_198'].values[0], 
                        'Status': 'OPEN'  # Updated to OPEN status
                    }, ignore_index=True)
                    
                    stocks_to_remove.append(symbol)
                    print(f"Bought {symbol} at {breakout_candle_close} on {current_date}")
            else:  # Previous days' breakout
                previous_day_high = details['highs'][str(next_trading_day(current_date - timedelta(days=1)))]
                breakout_candle = df_15min[df_15min['high'] > previous_day_high]
                if not breakout_candle.empty:
                    breakout_candle_close = breakout_candle.iloc[0]['close']
                    shares = np.floor(investment_per_stock / breakout_candle_close)
                    buy_value = round(shares * breakout_candle_close, 2)
                    summary_stats["Total Profit/Loss"] -= buy_value
                    
                    # Log the buy action
                    trade_log = trade_log.append({
                        'Symbol': symbol, 
                        'Entry Date': current_date, 
                        'Entry Time': breakout_candle.iloc[0]['date'], 
                        'Buy Price': breakout_candle_close, 
                        'Shares': shares, 
                        'Buy Value': buy_value, 
                        'Sell Date': np.nan, 
                        'Sell Price': np.nan, 
                        'Sell Value': np.nan, 
                        'Profit/Loss': np.nan, 
                        'ROI': np.nan, 
                        'High Value': breakout_candle.iloc[0]['high'], 
                        'Low Value': breakout_candle.iloc[0]['low'], 
                        'Max Drawdown': np.nan, 
                        'Exit Reason': np.nan, 
                        'SMA_198': current_data[current_data['Symbol'] == symbol]['SMA_198'].values[0], 
                        'Status': 'OPEN'  # Updated to OPEN status
                    }, ignore_index=True)
                    
                    stocks_to_remove.append(symbol)
                    print(f"Bought {symbol} at {breakout_candle_close} on {current_date}")

        # Decrease the days_to_check for each stock
        breakout_stocks_today[symbol]['days_to_check'] -= 1
        # If days_to_check reaches 0, remove the stock from the list
        if breakout_stocks_today[symbol]['days_to_check'] < 0:
            stocks_to_remove.append(symbol)

    # Remove bought stocks and those with expired days_to_check from the breakout_stocks_today
    for stock in stocks_to_remove:
        del breakout_stocks_today[stock]
    
    # Save updated breakout stocks
    with open(breakout_stocks_file, 'w') as f:
        json.dump(breakout_stocks_today, f, indent=4)

    # Mark trades as closed instead of removing
    to_update = []  # List to hold indices of trades to be updated
    for index, position in trade_log.iterrows():
        if pd.isnull(position['Sell Date']):
            current_price = current_data[current_data['Symbol'] == position['Symbol']]['close'].values[0]
            current_sma = current_data[current_data['Symbol'] == position['Symbol']]['SMA_198'].values[0]
            position['High Value'] = max(position['High Value'], current_price)
            position['Low Value'] = min(position['Low Value'], current_price)
            drawdown = (position['High Value'] - current_price) / position['High Value']
            position['Max Drawdown'] = max(position['Max Drawdown'], drawdown)
    
            # Exit condition based on stop loss, SMA_198, or 25% drawdown from high value
            exit_reason = None
            if current_price <= position['Buy Price'] * (1 - stop_loss_percent / 100):
                exit_reason = 'SL'
            elif current_price <= current_sma:
                exit_reason = 'SMA'
            elif drawdown >= 0.25:
                exit_reason = 'Drawdown'
    
            if exit_reason:
                trade_log.at[index, 'Sell Date'] = current_date
                trade_log.at[index, 'Sell Price'] = current_price
                trade_log.at[index, 'Sell Value'] = round(current_price * position['Shares'], 2)
                trade_log.at[index, 'Profit/Loss'] = round(trade_log.at[index, 'Sell Value'] - position['Buy Value'], 2)
                trade_log.at[index, 'ROI'] = round((trade_log.at[index, 'Profit/Loss'] / position['Buy Value']) * 100, 2)
                trade_log.at[index, 'Exit Reason'] = exit_reason
                trade_log.at[index, 'Status'] = 'Closed'  # Update Status to Closed
                summary_stats["Total Profit/Loss"] += trade_log.at[index, 'Sell Value']
                to_update.append(index)
    
    # Update summary statistics
    total_trades = len(trade_log)
    winning_trades = len(trade_log[trade_log['Profit/Loss'] > 0])
    winning_percentage = (winning_trades / total_trades) * 100 if total_trades > 0 else 0.0
    
    summary_stats.update({
        "Total Trades": total_trades,
        "Winning Trades": winning_trades,
        "Winning Percentage (%)": winning_percentage,
        "Max Drawdown": trade_log['Max Drawdown'].max() if not trade_log.empty else 0.0,
    })

    # Save logs and summary statistics
    trade_log.to_csv(log_file, index=False)
    pd.DataFrame([summary_stats]).to_csv(summary_file, index=False)

    print("Daily routine completed and logs updated.")

# Function to run daily routine every 15 minutes during market hours
def run_daily_routine():
    # Define market hours (9:15 AM to 3:30 PM IST)
    market_open_time = datetime.now().replace(hour=9, minute=15, second=0, microsecond=0)
    market_close_time = datetime.now().replace(hour=15, minute=30, second=0, microsecond=0)

    # Schedule daily routine every 15 minutes during market hours
    while datetime.now() < market_close_time:
        if datetime.now() >= market_open_time:
            print(f"Running daily routine at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            daily_routine()
        time.sleep(300)  # Sleep for 900 seconds (15 minutes)

# Start the routine in a separate thread
routine_thread = threading.Thread(target=run_daily_routine)
routine_thread.start()

# Keep the main thread alive
while routine_thread.is_alive():
    time.sleep(1)

{'status': 'success', 'data': {'profile': {}}}
Running daily routine at 2024-07-26 10:22:18
Processing date: 2024-07-26
No instrument found for symbol IBULHSGFIN
Breakout stocks updated: {}
Daily routine completed and logs updated.
Running daily routine at 2024-07-26 10:28:52
Processing date: 2024-07-26
No instrument found for symbol IBULHSGFIN
Breakout stocks updated: {}
Daily routine completed and logs updated.
Running daily routine at 2024-07-26 10:34:56
Processing date: 2024-07-26
No instrument found for symbol IBULHSGFIN
Breakout stocks updated: {}
Daily routine completed and logs updated.
Running daily routine at 2024-07-26 10:41:08
Processing date: 2024-07-26
No instrument found for symbol IBULHSGFIN
Breakout stocks updated: {}
Daily routine completed and logs updated.
Running daily routine at 2024-07-26 10:47:23
Processing date: 2024-07-26
No instrument found for symbol IBULHSGFIN
Breakout stocks updated: {}
Daily routine completed and logs updated.
Running daily routine at 202